In [2]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import pandas as pd
import numpy as np
import requests

In [3]:
#scraping data from web
url = "https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00014.xml"
    
xml_data = requests.get(url).content

soup = BeautifulSoup(xml_data, "xml")

last_name = soup.find_all('last_name')
first_name = soup.find_all('first_name')
party = soup.find_all('party')
state = soup.find_all('state')
vote_cast = soup.find_all('vote_cast')
lis_member_id = soup.find_all('lis_member_id')

data = []

for i in range(len(last_name)):
  row = [last_name[i].getText(), first_name[i].getText(), party[i].getText(), state[i].getText(), vote_cast[i].getText(), lis_member_id[i].getText()]
  data.append(row)

df = pd.DataFrame(data, columns = ['last_name', 'first_name', 'party', 'state', 'vote_cast', 'lis_member_id'])

In [9]:
df.head()

,last_name,first_name,party,state,vote_cast,lis_member_id
0,Baldwin,Tammy,D,WI,Yea,S354
1,Barrasso,John,R,WY,Yea,S317
2,Bennet,Michael,D,CO,Yea,S330
3,Blackburn,Marsha,R,TN,Yea,S396
4,Blumenthal,Richard,D,CT,Yea,S341


In [5]:
df

,last_name,first_name,party,state,vote_cast,lis_member_id
0,Baldwin,Tammy,D,WI,Yea,S354
1,Barrasso,John,R,WY,Yea,S317
2,Bennet,Michael,D,CO,Yea,S330
3,Blackburn,Marsha,R,TN,Yea,S396
4,Blumenthal,Richard,D,CT,Yea,S341
...,...,...,...,...,...,...
95,Warren,Elizabeth,D,MA,Yea,S366
96,Whitehouse,Sheldon,D,RI,Yea,S316
97,Wicker,Roger,R,MS,Yea,S318
98,Wyden,Ron,D,OR,Yea,S247


1)  Ratio of number of rows to the number of columns in your dataset?

In [44]:
df.shape[0]/df.shape[1]

16.666666666666668

2) What's the total number of Yea votes from all senators whose name starts with "B"?

In [16]:
yea_start_with_b = df[(df.last_name.str.startswith('B')) & (df.vote_cast == 'Yea')]
yea_start_with_b

,last_name,first_name,party,state,vote_cast,lis_member_id
0,Baldwin,Tammy,D,WI,Yea,S354
1,Barrasso,John,R,WY,Yea,S317
2,Bennet,Michael,D,CO,Yea,S330
3,Blackburn,Marsha,R,TN,Yea,S396
4,Blumenthal,Richard,D,CT,Yea,S341
5,Blunt,Roy,R,MO,Yea,S342
6,Booker,Cory,D,NJ,Yea,S370
7,Boozman,John,R,AR,Yea,S343
9,Brown,Sherrod,D,OH,Yea,S307
10,Burr,Richard,R,NC,Yea,S300


In [17]:
yea_start_with_b.shape[0]

10

3) How many Senators have their names starting with W?

In [18]:
start_with_w = df[(df.last_name.str.startswith('W'))]
start_with_w

,last_name,first_name,party,state,vote_cast,lis_member_id
93,Warner,Mark,D,VA,Yea,S327
94,Warnock,Raphael,D,GA,Yea,S415
95,Warren,Elizabeth,D,MA,Yea,S366
96,Whitehouse,Sheldon,D,RI,Yea,S316
97,Wicker,Roger,R,MS,Yea,S318
98,Wyden,Ron,D,OR,Yea,S247


In [19]:
start_with_w.shape[0]

6

4) How many states have both Republican senators?  

In [27]:
both_republican_df = df[df.party == 'R']
both_republican_dict = dict(both_republican_df['state'].value_counts())
both_republican_dict

{'WY': 2,
 'IA': 2,
 'AL': 2,
 'FL': 2,
 'SD': 2,
 'AK': 2,
 'KY': 2,
 'KS': 2,
 'UT': 2,
 'OK': 2,
 'MS': 2,
 'SC': 2,
 'TN': 2,
 'NE': 2,
 'ID': 2,
 'ND': 2,
 'TX': 2,
 'LA': 2,
 'NC': 2,
 'IN': 2,
 'AR': 2,
 'MO': 2,
 'MT': 1,
 'WI': 1,
 'ME': 1,
 'WV': 1,
 'OH': 1,
 'PA': 1}

In [32]:
both_republican_list = [i for i in both_republican_dict.keys() if both_republican_dict[i] == 2]
len(both_republican_list)

22

5) How many democrats voted Nay?

In [55]:
len(df[(df.party == 'D') & (df.vote_cast == 'Nay')].value_counts())

0

6) How many states voted all Nay?

In [68]:
nay_states_dict = dict(df[df.vote_cast == 'Nay'].state.value_counts())
all_nay_states_list = [i for i in nay_states_dict.keys() if nay_states_dict[i] == 2]
len(all_nay_states_list)

0

7)  What is the highest absolute difference in percentage between yea and nay senators among the different parties?

In [65]:
nay_dems = len(df[(df.party == 'D') & (df.vote_cast == 'Nay')].value_counts())
yea_dems = len(df[(df.party == 'D') & (df.vote_cast == 'Yea')].value_counts())
dems_absolute_diff = abs(nay_dems - yea_dems)/(nay_dems + yea_dems)
print("Absolute difference between yea and nay for democrats:",dems_absolute_diff)

nay_reps = len(df[(df.party == 'R') & (df.vote_cast == 'Nay')].value_counts())
yea_reps = len(df[(df.party == 'R') & (df.vote_cast == 'Yea')].value_counts())
reps_absolute_diff = abs(nay_reps - yea_reps)/(nay_reps + yea_reps)
print("Absolute difference between yea and nay for republicans: ",reps_absolute_diff)

print("Highest absolute difference in percentage: ", end="")
print(dems_absolute_diff*100 if dems_absolute_diff > reps_absolute_diff else reps_absolute_diff*100)

Absolute difference between yea and nay for democrats: 1.0
Absolute difference between yea and nay for republicans:  0.6
Highest absolute difference in percentage: 100.0


8) What is the votes ratio of Democrats' major decision to Republicans' major decision

In [67]:
dems_major = nay_dems if nay_dems > yea_dems else yea_dems
reps_major = nay_reps if nay_dems > yea_reps else yea_reps
dems_major/reps_major

1.2

9) How many states had at least 1 senator voting Nay?

In [69]:
len(set(nay_states_dict.keys()))

10

10) How many states have both Republican and Democrat senators?

In [83]:
state_with_dems = set(df[df.party == 'D'].state)
state_with_reps = set(df[df.party == 'R'].state)
print("States with both Republican and Democrat senators: ",state_with_dems & state_with_reps)
len(state_with_dems & state_with_reps)

States with both Republican and Democrat senators:  {'WI', 'WV', 'PA', 'OH', 'MT'}


5